# T-Maze Inference for Planning with GFE

This notebook reports the GFE for all policies for a variety of scenarios.

In [43]:
using LinearAlgebra
using ForneyLab
using Plots

import ForneyLab: collectNaiveVariationalNodeInbounds

# T-maze layout
# [2| |3]
#   | |
#   |1|
#   |4|

include("factor_nodes/GFECategorical.jl")
include("update_rules/GFECategorical.jl")

# Model

In [44]:
T = 2

fg = FactorGraph()

u = Vector{Variable}(undef, T)
x = Vector{Variable}(undef, T)

@RV x_t_min ~ Categorical(placeholder(:D_t_min, dims=(8,)))

x_k_min = x_t_min
for k=1:T
    @RV u[k]
    @RV x[k] ~ Transition(x_k_min, u[k])

    placeholder(u[k], :u, index=k, dims=(8,8))
    GFECategorical(x[k], 
                   placeholder(:A, dims=(16,8), var_id=:A_*k), 
                   placeholder(:C, dims=(16,), index=k, var_id=:C_*k))
    
    x_k_min = x[k]
end
;

In [45]:
q = PosteriorFactorization(fg)
algo = messagePassingAlgorithm(q, free_energy=true)
code = algorithmSourceCode(algo, free_energy=true)
eval(Meta.parse(code))
;

# Results

In [48]:
# Reward probability and utility, uncomment scenario of interest
α = 0.9; c = 2.0
# α = 0.5; c = 2.0
# α = 0.9; c = 0.0
# α = 1.0; c = 2.0

include("environment.jl")
include("agent.jl")
include("helpers.jl")

(A, B, C, D) = constructABCD(α, c)
;

## Constrained results

In [49]:
GFE = evaluatePolicies(A, B, C, D)
plotResults(CBFE, clim=(7.0,12.0), dpi=300, highlight=minimum)
savefig("figures/GFE_c_$(c)_a_$(α).png")